In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset, Subset

In [29]:
data = np.load('./Mnist_10000._samples.npy')

In [30]:
# Convert the structured array to PyTorch tensors
images = torch.tensor([item['image'] for item in data], dtype=torch.float32)
labels = torch.tensor([item['label'] for item in data], dtype=torch.long)

# Flatten the images for a simple fully connected network
images = images.view(images.shape[0], -1)


# Create a dataset and data loader
dataset = TensorDataset(images, labels)


In [31]:
class_indices = {k: [] for k in range(10)}  # Assuming 10 classes (0-9)
for idx, (image, label) in enumerate(dataset):
    class_indices[label.item()].append(idx)

# Step 2: Randomly select 10 samples from each class
labeled_indices = []
for indices in class_indices.values():
    labeled_indices.extend(np.random.choice(indices, 10, replace=False))

# Create a mask for the rest of the data for testing
mask = np.ones(len(dataset), dtype=bool)
mask[labeled_indices] = False
unlabeled_indices = np.arange(len(dataset))[mask]

# Step 3: Create training and testing subsets
labeled_set = Subset(dataset, labeled_indices)
unlabeled_set = Subset(dataset, unlabeled_indices)

In [38]:
# Step 4: Create DataLoaders
labeledLoader = DataLoader(labeled_set, batch_size=10, shuffle=True)
unlabeledLoader = DataLoader(unlabeled_set, batch_size=50, shuffle=False)


In [39]:
class Net(nn.Module):

    def __init__(self):

        super(Net,self).__init__()

        self.fullyConnectedLayer = nn.Sequential(
            nn.Linear(784, 200),
            nn.ReLU(),
            nn.Linear(200,10)
        )

    def forward(self, input):

        output = self.fullyConnectedLayer(input)
        activatedOutput = F.log_softmax(output, dim = 1)

        return activatedOutput
    

In [57]:
def kl_divergence_with_logits(p_logits, q_logits):
    p = F.softmax(p_logits, dim=1)
    log_p = F.log_softmax(p_logits, dim=1)
    log_q = F.log_softmax(q_logits, dim=1)
    kl = torch.sum(p * (log_p - log_q), dim=1)
    return kl.mean()

def virtual_adversarial_loss(model, x, epsilon=0.02):
    x.requires_grad_()
    with torch.enable_grad():
        logits = model(x)
        logits_perturbed = model(x + epsilon * torch.randn_like(x))
        loss = kl_divergence_with_logits(logits, logits_perturbed)
    return loss

In [54]:
model1 = Net()

In [58]:
import torch.optim as optim

optimizer1 = optim.SGD(model1.parameters(), lr=0.01)
optimizer2 = optim.SGD(model1.parameters(), lr = 0.001)
num_epochs = 50
supervised_loss = 0
VAT_loss = 0
total_loss = 0

for epoch in range(num_epochs):
    model1.train()
    
    # Train on labeled data
    for images, labels in labeledLoader:
        optimizer1.zero_grad()
        outputs = model1(images)
        supervised_loss = F.cross_entropy(outputs, labels)
        VAT_loss_labeled = virtual_adversarial_loss(model1, images, epsilon=0.02)
        total_loss = supervised_loss + VAT_loss_labeled
        total_loss.backward()
        optimizer1.step()

    print(f"For Epoch{epoch} SuperVised_Loss:{supervised_loss},       VAT_loss = {VAT_loss_labeled},   Total_Loss:{total_loss}")
    
    # Train on unlabeled data
    for images, labels in unlabeledLoader:

        optimizer2.zero_grad()

        vat_loss_unlabeled = virtual_adversarial_loss(model1, images, epsilon=0.02)
        vat_loss_unlabeled.backward()
        optimizer2.step()

    
    print(f"For Epoch{epoch} VAT_LOSS_UNLABELED:{vat_loss_unlabeled}")

For Epoch0 SuperVised_Loss:2.212797164916992,       VAT_loss = 1.9250153854954988e-05,   Total_Loss:2.2128164768218994
For Epoch0 VAT_LOSS_UNLABELED:1.7019729057210498e-05
For Epoch1 SuperVised_Loss:2.191187620162964,       VAT_loss = 1.8329688828089274e-05,   Total_Loss:2.1912059783935547
For Epoch1 VAT_LOSS_UNLABELED:1.70345556398388e-05
For Epoch2 SuperVised_Loss:2.2240986824035645,       VAT_loss = 2.2615375200985e-05,   Total_Loss:2.224121332168579
For Epoch2 VAT_LOSS_UNLABELED:1.8634174921317026e-05
For Epoch3 SuperVised_Loss:2.0205984115600586,       VAT_loss = 3.0069513741182163e-05,   Total_Loss:2.0206284523010254
For Epoch3 VAT_LOSS_UNLABELED:1.8652455764822662e-05
For Epoch4 SuperVised_Loss:1.8592243194580078,       VAT_loss = 1.5717281712568365e-05,   Total_Loss:1.8592400550842285
For Epoch4 VAT_LOSS_UNLABELED:2.0284278434701264e-05
For Epoch5 SuperVised_Loss:2.235863208770752,       VAT_loss = 1.7860696971183643e-05,   Total_Loss:2.2358810901641846
For Epoch5 VAT_LOSS_UNLA

In [59]:
def calculate_accuracy(loader,model):
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()  

    return 100 * correct / total

train_accuracy = calculate_accuracy(labeledLoader, model1)
test_accuracy = calculate_accuracy(unlabeledLoader, model1)

print(f'Training Accuracy: {train_accuracy}%')
print(f'Testing Accuracy: {test_accuracy}%')

Training Accuracy: 75.0%
Testing Accuracy: 16.01010101010101%


### VAT for Two Moons Dataset

In [60]:
selected_samples = np.load('./selected_samples.npy')

remaining_samples = np.load('./remaining_dataset.npy')

# Converting the selected samples and remaining samples into PyTorch tensors
selected_samples_tensor = torch.tensor(selected_samples, dtype=torch.float32)
remaining_samples_tensor = torch.tensor(remaining_samples, dtype=torch.float32)

# Extracting features and labels for both datasets
features_selected = selected_samples_tensor[:, :2]
labels_selected = selected_samples_tensor[:, 2].long()  # converting labels to long for classification

features_remaining = remaining_samples_tensor[:, :2]
labels_remaining = remaining_samples_tensor[:, 2].long()

# Creating TensorDatasets
selected_dataset = TensorDataset(features_selected, labels_selected)
remaining_dataset = TensorDataset(features_remaining, labels_remaining)

# Creating DataLoaders
selected_loader = DataLoader(selected_dataset, batch_size=1)  # small batch size for the small dataset
remaining_loader = DataLoader(remaining_dataset, batch_size=10)  # larger batch size for the larger dataset

In [61]:
class TwoMoonsNet(nn.Module):

    def __init__(self):

        super(TwoMoonsNet,self).__init__()

        self.fullyConnectedLayer = nn.Sequential(
            nn.Linear(2, 10),
            nn.ReLU(),
            nn.Linear(10,2)
        )

    def forward(self, input):

        output = self.fullyConnectedLayer(input)
        activatedOutput = F.log_softmax(output, dim = 1)

        return activatedOutput

In [65]:
model2 = TwoMoonsNet()

In [68]:
import torch.optim as optim

optimizer1 = optim.SGD(model2.parameters(), lr=0.01)
optimizer2 = optim.SGD(model2.parameters(), lr = 0.001)
num_epochs = 100
supervised_loss = 0
VAT_loss = 0
total_loss = 0

for epoch in range(num_epochs):
    model2.train()
    
    # Train on labeled data
    for images, labels in selected_loader:
        optimizer1.zero_grad()
        outputs = model2(images)
        supervised_loss = F.cross_entropy(outputs, labels)
        VAT_loss_labeled = virtual_adversarial_loss(model2, images, epsilon=0.02)
        total_loss = supervised_loss + VAT_loss_labeled
        total_loss.backward()
        optimizer1.step()

    print(f"For Epoch{epoch} SuperVised_Loss:{supervised_loss},       VAT_loss = {VAT_loss_labeled},   Total_Loss:{total_loss}")
    
    # Train on unlabeled data
    for images, labels in remaining_loader:

        optimizer2.zero_grad()

        vat_loss_unlabeled = virtual_adversarial_loss(model2, images, epsilon=0.02)
        vat_loss_unlabeled.backward()
        optimizer2.step()

    
    print(f"For Epoch{epoch} VAT_LOSS_UNLABELED:{vat_loss_unlabeled}")

For Epoch0 SuperVised_Loss:0.13327543437480927,       VAT_loss = 1.1388328857719898e-07,   Total_Loss:0.13327555358409882
For Epoch0 VAT_LOSS_UNLABELED:9.214194142259657e-05
For Epoch1 SuperVised_Loss:0.1309199035167694,       VAT_loss = 3.628828562796116e-05,   Total_Loss:0.13095618784427643
For Epoch1 VAT_LOSS_UNLABELED:1.4502278645522892e-05
For Epoch2 SuperVised_Loss:0.12861710786819458,       VAT_loss = 5.523033905774355e-07,   Total_Loss:0.12861765921115875
For Epoch2 VAT_LOSS_UNLABELED:0.0004363400803413242
For Epoch3 SuperVised_Loss:0.12636582553386688,       VAT_loss = 2.238526940345764e-05,   Total_Loss:0.12638820707798004
For Epoch3 VAT_LOSS_UNLABELED:0.0001563768310006708
For Epoch4 SuperVised_Loss:0.12416380643844604,       VAT_loss = 2.7777859941124916e-06,   Total_Loss:0.1241665855050087
For Epoch4 VAT_LOSS_UNLABELED:6.358710379572585e-06
For Epoch5 SuperVised_Loss:0.12201178818941116,       VAT_loss = 1.6749254427850246e-08,   Total_Loss:0.12201180309057236
For Epoch5 V

In [69]:
train_accuracy = calculate_accuracy(selected_loader, model2)
test_accuracy = calculate_accuracy(remaining_loader, model2)

print(f'Training Accuracy: {train_accuracy}%')
print(f'Testing Accuracy: {test_accuracy}%')

Training Accuracy: 100.0%
Testing Accuracy: 72.34042553191489%
